In [1]:
import csv

In [2]:
import imageio
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
model_dir = 'data/models/test3/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

### Load from CSV

In [4]:
def load_images(data_dir, valid_cameras, discard_prob):
    df = pd.read_csv(data_dir+'driving_log.csv')
    images = []
    measurements = []

    def fetch_image(data_dir, source_path):
        filename = source_path.split('/')[-1]
        current_path = data_dir + 'IMG/' + filename
        return imageio.imread(current_path)

    def append_image(image, measurement):
        images.append(image)
        measurements.append(measurement)

        # flip image
        image_flipped = np.fliplr(image)
        measurement_flipped = -measurement
        images.append(image_flipped)
        measurements.append(measurement_flipped)

    for idx, row in df.iterrows():
        measurement = float(row[3])
        camera_map = { 1:0.25, 0:0, 2:-0.3 }
        if measurement <= 0 and np.random.rand() < discard_prob:
            continue
        for c in valid_cameras:
            image = fetch_image(data_dir, row[c])
            append_image(image, measurement + camera_map[c])
            
    return images, measurements
    
def load_from_dir(data_dir, discard_prob=0, valid_cameras=[0, 1, 2], load_cached=False):
    if load_cached and os.path.exists(data_dir+'X_train.npy'):
        X_train = np.load(data_dir+'X_train.npy')
        y_train = np.load(data_dir+'y_train.npy')
    else:
        images, measurements = load_images(data_dir, valid_cameras, discard_prob)
        
        X_train = np.array(images)
        y_train = np.array(measurements)

        np.save(data_dir+'X_train.npy', X_train)
        np.save(data_dir+'y_train.npy', y_train)
    
    print('X_train:', X_train.shape)
    print('y_train:', y_train.shape)
    return X_train, y_train
    

In [5]:
X_train, y_train = load_from_dir('data/sample_data/', discard_prob=0.5, load_cached=False)

X_train: (29898, 160, 320, 3)
y_train: (29898,)


### Select images from each bucket for testing

In [ ]:
y_train[:30]

In [ ]:
print(y_train[2])
plt.imshow(X_train[2])

### Create Model

In [6]:
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, BatchNormalization, Input

Using TensorFlow backend.


### NVIDIA Suggested network

In [7]:
model = Sequential()
model.add(Lambda(lambda x: x - 255.0 / 255.0, input_shape=(160,320,3)))
# model.add(Lambda(lambda x: x - 255.0 / 255.0, input_shape=(66,200,3))) # Checking with NVidia dimensions
model.add(keras.layers.Convolution2D(24, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(36, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(48, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(64, (3,3), strides=(1,1), activation='relu'))
model.add(keras.layers.Convolution2D(64, (3,3), strides=(1,1), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=.001))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 78, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 17, 37, 48)        43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 35, 64)        27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 33, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 27456)             0         
__________

In [9]:

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save(model_dir+'nvidia_suggested_model.h5')

Train on 23918 samples, validate on 5980 samples
Epoch 1/5
23918/23918 [==============================] - 28s - loss: 7.4662 - val_loss: 0.0173
Epoch 2/5
23918/23918 [==============================] - 26s - loss: 0.0132 - val_loss: 0.0151
Epoch 3/5
23918/23918 [==============================] - 26s - loss: 0.0119 - val_loss: 0.0137
Epoch 4/5
23918/23918 [==============================] - 26s - loss: 0.0110 - val_loss: 0.0136
Epoch 5/5
23918/23918 [==============================] - 26s - loss: 0.0107 - val_loss: 0.0131


In [10]:
# model = keras.models.load_model(model_dir+'nvidia_suggested_model.h5')
model.optimizer.lr.assign(0.005)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=4)

model.save(model_dir+'nvidia_suggested_model_2.h5')

Train on 23918 samples, validate on 5980 samples
Epoch 1/4
23918/23918 [==============================] - 26s - loss: 0.0103 - val_loss: 0.0140
Epoch 2/4
23918/23918 [==============================] - 26s - loss: 0.0101 - val_loss: 0.0133
Epoch 3/4
23918/23918 [==============================] - 26s - loss: 0.0097 - val_loss: 0.0141
Epoch 4/4
23918/23918 [==============================] - 26s - loss: 0.0094 - val_loss: 0.0142


In [11]:
# model = keras.models.load_model(model_dir+'nvidia_suggested_model_2.h5')
model.optimizer.lr.assign(0.0001)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save(model_dir+'nvidia_suggested_model_3.h5')

Train on 23918 samples, validate on 5980 samples
Epoch 1/3
23918/23918 [==============================] - 26s - loss: 0.0094 - val_loss: 0.0142
Epoch 2/3
23918/23918 [==============================] - 26s - loss: 0.0091 - val_loss: 0.0134
Epoch 3/3
23918/23918 [==============================] - 26s - loss: 0.0089 - val_loss: 0.0149


### Dirt roads

In [13]:
X_train_dirt, y_train_dirt = load_from_dir('data/recorded_data/lap1_dirt_bridge/', discard_prob=0, load_cached=False)
# model = keras.models.load_model('data/sample_data/'+'nvidia_suggested_model.h5')

X_train: (3468, 160, 320, 3)
y_train: (3468,)


In [ ]:
plt.imshow(X_train_dirt[2200])
print(y_train_dirt[2200])

In [14]:
model.optimizer.lr.assign(0.0001)
model.fit(X_train_dirt, y_train_dirt, validation_split=0.1, shuffle=True, epochs=7)

model.save(model_dir+'nvidia_suggested_model_dirt.h5')

Train on 3121 samples, validate on 347 samples
Epoch 1/7
3121/3121 [==============================] - 3s - loss: 0.0056 - val_loss: 0.0117
Epoch 2/7
3121/3121 [==============================] - 3s - loss: 0.0019 - val_loss: 0.0123
Epoch 3/7
3121/3121 [==============================] - 3s - loss: 0.0013 - val_loss: 0.0120
Epoch 4/7
3121/3121 [==============================] - 3s - loss: 0.0011 - val_loss: 0.0117
Epoch 5/7
3121/3121 [==============================] - 3s - loss: 7.2086e-04 - val_loss: 0.0117
Epoch 6/7
3121/3121 [==============================] - 3s - loss: 6.2248e-04 - val_loss: 0.0116
Epoch 7/7
3121/3121 [==============================] - 3s - loss: 5.3836e-04 - val_loss: 0.0117


In [17]:
model.optimizer.lr.assign(0.00005)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=2)

model.save(model_dir+'nvidia_suggested_model_dirt2.h5')

Train on 23918 samples, validate on 5980 samples
Epoch 1/2
23918/23918 [==============================] - 26s - loss: 0.0084 - val_loss: 0.0139
Epoch 2/2
23918/23918 [==============================] - 26s - loss: 0.0078 - val_loss: 0.0137


### Additional data

In [ ]:
X_train, y_train = load_from_dir('data/recorded_data/lap1/', [0, 1, 2], False)

In [ ]:
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

model.save(model_dir+'nvidia_suggested_model_self_lap1.h5')

### Lap1 Mistakes

In [ ]:
X_train, y_train = load_from_dir('data/recorded_data/lap1mistakes/', [0, 1, 2], False)

In [ ]:
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=6)

model.save(model_dir+'nvidia_suggested_model_self_lap1mistakes.h5')

### Lap1 Reverse

In [ ]:
X_train, y_train = load_from_dir('data/recorded_data/lap1reverse/', [0, 1, 2], False)

In [ ]:
# model.optimizer.lr = 0.001
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save(model_dir+'nvidia_suggested_model_self_lap1reverse.h5')